In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from typing import List
from tqdm import tqdm


%matplotlib inline

import catalyst 
import recbole

from typing import Dict, List, Tuple

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.nn.init import constant_, xavier_normal_
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset

from catalyst import dl, metrics
from catalyst.contrib.datasets import MovieLens
from catalyst.utils import get_device, set_global_seed
from torch.nn.utils.rnn import pad_sequence 

import random

from recbole.model.abstract_recommender import SequentialRecommender
from recbole.model.layers import *


set_global_seed(100)

### ML-100k

In [ ]:
df = pd.read_csv('data/ML_100k.csv')
df = df.rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df['weekday'] = pd.to_datetime(df.timestamp).dt.weekday
df['hour'] = pd.to_datetime(df.timestamp).dt.hour
df.head()

,user_id,item_id,rating,timestamp,weekday,hour
0,1,31,2.5,2009-12-14 02:52:24,0,2
1,1,1029,3.0,2009-12-14 02:52:59,0,2
2,1,1061,3.0,2009-12-14 02:53:02,0,2
3,1,1129,2.0,2009-12-14 02:53:05,0,2
4,1,1172,4.0,2009-12-14 02:53:25,0,2


In [ ]:
splitter = RandomSplit(test_fraction=0.2)
train_df, valid_df, test_df = splitter(df)

In [ ]:
train_grouped = train_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id, 
                                                                 x.timestamp,
                                                                 x.weekday,
                                                                 x.hour), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


train_grouped.head()

,user_id,train_interactions
0,1,"[(2294, 2009-12-14 02:51:48, 0, 2), (2455, 200..."
1,2,"[(150, 1996-06-21 11:09:55, 4, 11), (296, 1996..."
2,3,"[(355, 2011-02-28 02:53:09, 0, 2), (1271, 2011..."
3,4,"[(1210, 2000-02-05 19:25:14, 5, 19), (2734, 20..."
4,5,"[(1380, 2006-11-12 23:10:44, 6, 23), (1035, 20..."


In [ ]:
joined = train_grouped.merge(valid_grouped).merge(test_grouped)
joined.head()

,user_id,train_interactions,valid_interactions,test_interactions
0,1,"[(2294, 2009-12-14 02:51:48, 0, 2), (2455, 200...","[(1029, 2009-12-14 02:52:59, 0, 2), (1061, 200...","[(1953, 2009-12-14 02:53:11, 0, 2), (2150, 200..."
1,2,"[(150, 1996-06-21 11:09:55, 4, 11), (296, 1996...","[(168, 1996-06-21 11:15:10, 4, 11), (377, 1996...","[(585, 1996-06-21 11:16:57, 4, 11), (474, 1996..."
2,3,"[(355, 2011-02-28 02:53:09, 0, 2), (1271, 2011...","[(1884, 2011-02-28 03:19:03, 0, 3), (778, 2011...","[(58559, 2011-02-28 19:41:11, 0, 19), (6377, 2..."
3,4,"[(1210, 2000-02-05 19:25:14, 5, 19), (2734, 20...","[(1674, 2000-02-07 04:04:04, 0, 4), (1127, 200...","[(910, 2000-02-07 10:28:26, 0, 10), (2080, 200..."
4,5,"[(1380, 2006-11-12 23:10:44, 6, 23), (1035, 20...","[(5669, 2006-11-12 23:30:06, 6, 23), (33166, 2...","[(586, 2006-11-12 23:33:12, 6, 23), (150, 2006..."


In [ ]:
our_items = set()
for idx, row in tqdm(joined.iterrows()):
    for el in row.train_interactions:
        our_items.add(el[0])
        
len(our_items)





671it [00:00, 25578.51it/s]


5989

In [ ]:
item2idx = {k: i for i, k in enumerate(our_items)}
idx2item = {i: k for k, i in item2idx.items()}

In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, ds, num_items, item2idx, phase='valid', N=200):
        super().__init__()
        self.ds = ds
        self.phase = phase
        self.n_items = num_items
        self.item2idx = item2idx
        self.N = N 
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        
        row = self.ds.iloc[idx]
        
        x_input = np.zeros(self.n_items+1)
        x_input[[self.item2idx[x[0]]+1 for x in row['train_interactions'] if x[0] in self.item2idx]] = 1
        
        days_of_weeks = [x[2] for x in row['train_interactions'] if x[0] in self.item2idx][-self.N+1:]
        
        hours = [x[3] for x in row['train_interactions'] if x[0] in self.item2idx][-self.N+1:]
        
        seq_input = [self.item2idx[x[0]]+1 for x in row['train_interactions'] if x[0] in self.item2idx][-self.N+1:]
        
        targets = np.zeros(self.n_items+1)
        
        dow_valid = row['valid_interactions'][0][2]
        dow_test = row['test_interactions'][0][2]
        
        hours_valid = row['valid_interactions'][0][3]
        hours_test = row['test_interactions'][0][3]
        
        if self.phase == 'train':
            return (seq_input, days_of_weeks, hours, dow_valid, hours_valid)
        elif self.phase == 'valid':
            targets[[self.item2idx[x[0]]+1 for x in row['valid_interactions'] if x[0] in self.item2idx]] = 1
        else:
            return (seq_input, days_of_weeks, hours, dow_test, hours_test)
            
        return (targets, seq_input, days_of_weeks, hours, dow_valid, hours_valid)
     
n_items = len(item2idx)

train = MyDataset(ds=joined,
                  num_items=n_items, 
                  item2idx=item2idx,
                  phase='train')

valid = MyDataset(ds=joined,
                  num_items=n_items,
                  item2idx=item2idx,
                  phase='valid')

print(len(train),len(valid))

671 671


In [ ]:
def collate_fn_train(batch: List[Tuple[torch.Tensor]]) -> Dict[str, torch.Tensor]: 
    
    seq_i,days_of_weeks,hours,dow_valid,hours_valid = zip(*batch)
    seq_len = torch.Tensor([len(x) for x in seq_i])
    dow_valid = torch.Tensor([x for x in dow_valid])
    hours_valid = torch.Tensor([x for x in hours_valid])
    seq_i = pad_sequence([torch.Tensor(t) for t in seq_i]).T    
    days_of_weeks = pad_sequence([torch.Tensor(t) for t in days_of_weeks]).T
    hours = pad_sequence([torch.Tensor(t) for t in hours]).T
    
    return {'seq_i': seq_i, 
            'seq_len':seq_len,
            'dow': days_of_weeks,
            'hours': hours,
            'dow_valid': dow_valid,
            'hours_valid': hours_valid}


def collate_fn_valid(batch: List[Tuple[torch.Tensor]]) -> Dict[str, torch.Tensor]:
    
    y, seq_i, days_of_weeks, hours, dow_valid, hours_valid = zip(*batch)
    
    seq_len = torch.Tensor([len(x) for x in seq_i]).long()
    seq_i = pad_sequence([torch.Tensor(t) for t in seq_i]).T.long()
    days_of_weeks = pad_sequence([torch.Tensor(t) for t in days_of_weeks]).T.long()
    hours = pad_sequence([torch.Tensor(t) for t in hours]).T.long()
    dow_valid = torch.Tensor([x for x in dow_valid])
    hours_valid = torch.Tensor([x for x in hours_valid])
            
    targets = pad_sequence([torch.Tensor(t) for t in y]).T

    return {"targets": targets,
            'seq_i': seq_i,
            'seq_len':seq_len,
            'dow': days_of_weeks,
            'hours': hours,
            'dow_valid': dow_valid,
            'hours_valid': hours_valid}

In [ ]:
loaders = {
        "train": DataLoader(train, batch_size=256, collate_fn=collate_fn_train),
        "valid": DataLoader(valid, batch_size=256, collate_fn=collate_fn_valid),
}

In [ ]:
model = ContextBERT4Rec(n_items=len(item2idx)+1, mask_ratio=0.2, hidden_size=64)

optimizer = optim.Adam(model.parameters(), lr=0.001)
lr_scheduler = StepLR(optimizer, step_size=20, gamma=0.1)
engine = dl.DeviceEngine('cpu')
hparams = {
    "anneal_cap": 0.2,
    "total_anneal_steps": 6000,
}


callbacks = [
    dl.NDCGCallback("logits", "targets", [20]),
    dl.MAPCallback("logits", "targets", [10]),
    dl.OptimizerCallback("loss", accumulation_steps=1),
    dl.EarlyStoppingCallback(
        patience=5, loader_key="valid", metric_key="map10", minimize=False
    )
]


runner = RecSysRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    engine=engine,
    hparams=hparams,
    scheduler=lr_scheduler,
    loaders=loaders,
    num_epochs=100,
    verbose=True,
    timeit=True,
    callbacks=callbacks,
)


test_dataset = MyDataset(ds=joined, num_items=n_items, phase='test',item2idx=item2idx)


inference_loader = DataLoader(test_dataset, 
                              batch_size=joined.shape[0]//100, 
                              collate_fn=collate_fn_train,)

preds = []

for prediction in tqdm(runner.predict_loader(loader=inference_loader)):
    preds.extend(prediction.detach().cpu().numpy().tolist())
    
print(len(preds))
assert len(preds) == joined.shape[0]

joined['preds_contextbert4rec'] = preds
joined['recs_contextbert4rec_10'] = joined['preds_contextbert4rec'].apply(lambda x: np.argsort(-np.array(x))[:10])
joined['recs_contextbert4rec_10'] = joined['recs_contextbert4rec_10'].apply(lambda x: [idx2item[t-1] for t in x])
joined['recs_contextbert4rec_5'] = joined['preds_contextbert4rec'].apply(lambda x: np.argsort(-np.array(x))[:5])
joined['recs_contextbert4rec_5'] = joined['recs_contextbert4rec_5'].apply(lambda x: [idx2item[t-1] for t in x])
joined.drop(['preds_contextbert4rec'],axis=1, inplace=True)






1/100 * Epoch (train):   0%|          | 0/3 [00:00<?, ?it/s]



1/100 * Epoch (train):   0%|          | 0/3 [00:01<?, ?it/s, _timer/_fps=351.534, _timer/batch_time=0.728, _timer/data_time=0.028, _timer/model_time=0.701, loss=8.701, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]



1/100 * Epoch (train):  33%|███▎      | 1/3 [00:01<00:02,  1.17s/it, _timer/_fps=351.534, _timer/batch_time=0.728, _timer/data_time=0.028, _timer/model_time=0.701, loss=8.701, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]



1/100 * Epoch (train):  33%|███▎      | 1/3 [00:02<00:02,  1.17s/it, _timer/_fps=235.942, _timer/batch_time=1.085, _timer/data_time=0.478, _timer/model_time=0.607, loss=8.679, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]



1/100 * Epoch (train):  67%|██████▋   | 2/3 [00:02<00:01,  1.15s/it, _timer/_fps=235.942, _timer/batch_time=1.085, _timer/data_time=0.478, _timer/model_time=0.607, loss=8.679, lr=1.000e-03, map10=0.000e+00, m

train (1/100) loss: 8.680811002250932 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0






1/100 * Epoch (valid):   0%|          | 0/3 [00:00<?, ?it/s]



1/100 * Epoch (valid):   0%|          | 0/3 [00:00<?, ?it/s, _timer/_fps=479.889, _timer/batch_time=0.533, _timer/data_time=0.038, _timer/model_time=0.495, loss=8.533, lr=1.000e-03, map10=0.055, momentum=0.900, ndcg20=0.026]



1/100 * Epoch (valid):  33%|███▎      | 1/3 [00:00<00:01,  1.88it/s, _timer/_fps=479.889, _timer/batch_time=0.533, _timer/data_time=0.038, _timer/model_time=0.495, loss=8.533, lr=1.000e-03, map10=0.055, momentum=0.900, ndcg20=0.026]



1/100 * Epoch (valid):  33%|███▎      | 1/3 [00:01<00:01,  1.88it/s, _timer/_fps=488.231, _timer/batch_time=0.524, _timer/data_time=0.038, _timer/model_time=0.486, loss=8.567, lr=1.000e-03, map10=0.056, momentum=0.900, ndcg20=0.027]



1/100 * Epoch (valid):  67%|██████▋   | 2/3 [00:01<00:00,  1.89it/s, _timer/_fps=488.231, _timer/batch_time=0.524, _timer/data_time=0.038, _timer/model_time=0.486, loss=8.567, lr=1.000e-03, map10=0.056, momentum=0.900, ndcg20=0.027]

valid (1/100) loss: 8.558565723913793 | lr: 0.001 | map10: 0.05718392631158388 | map10/std: 0.003472161154679869 | momentum: 0.9 | ndcg20: 0.026292348416569335 | ndcg20/std: 0.0005414539552975022
* Epoch (1/100) 






2/100 * Epoch (train):   0%|          | 0/3 [00:00<?, ?it/s]



2/100 * Epoch (train):   0%|          | 0/3 [00:01<?, ?it/s, _timer/_fps=367.875, _timer/batch_time=0.696, _timer/data_time=0.029, _timer/model_time=0.667, loss=8.595, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]



2/100 * Epoch (train):  33%|███▎      | 1/3 [00:01<00:02,  1.21s/it, _timer/_fps=367.875, _timer/batch_time=0.696, _timer/data_time=0.029, _timer/model_time=0.667, loss=8.595, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]



2/100 * Epoch (train):  33%|███▎      | 1/3 [00:02<00:02,  1.21s/it, _timer/_fps=194.151, _timer/batch_time=1.319, _timer/data_time=0.550, _timer/model_time=0.769, loss=8.565, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]



2/100 * Epoch (train):  67%|██████▋   | 2/3 [00:02<00:01,  1.29s/it, _timer/_fps=194.151, _timer/batch_time=1.319, _timer/data_time=0.550, _timer/model_time=0.769, loss=8.565, lr=1.000e-03, map10=0.000e+00, m

...

train (17/100) loss: 7.620365115818075 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0






17/100 * Epoch (valid):   0%|          | 0/3 [00:00<?, ?it/s]



17/100 * Epoch (valid):   0%|          | 0/3 [00:00<?, ?it/s, _timer/_fps=521.741, _timer/batch_time=0.491, _timer/data_time=0.034, _timer/model_time=0.456, loss=7.488, lr=1.000e-03, map10=0.178, momentum=0.900, ndcg20=0.106]



17/100 * Epoch (valid):  33%|███▎      | 1/3 [00:00<00:00,  2.04it/s, _timer/_fps=521.741, _timer/batch_time=0.491, _timer/data_time=0.034, _timer/model_time=0.456, loss=7.488, lr=1.000e-03, map10=0.178, momentum=0.900, ndcg20=0.106]



17/100 * Epoch (valid):  33%|███▎      | 1/3 [00:00<00:00,  2.04it/s, _timer/_fps=537.663, _timer/batch_time=0.476, _timer/data_time=0.033, _timer/model_time=0.443, loss=7.696, lr=1.000e-03, map10=0.186, momentum=0.900, ndcg20=0.111]



17/100 * Epoch (valid):  67%|██████▋   | 2/3 [00:00<00:00,  2.07it/s, _timer/_fps=537.663, _timer/batch_time=0.476, _timer/data_time=0.033, _timer/model_time=0.443, loss=7.696, lr=1.000e-03, map10=0.186, momentum=0.900, ndcg20=0

valid (17/100) loss: 7.604773068037189 | lr: 0.001 | map10: 0.1905959633618993 | map10/std: 0.015929656392303683 | momentum: 0.9 | ndcg20: 0.10844120066807274 | ndcg20/std: 0.0021501530854073254
* Epoch (17/100) 






0it [00:00, ?it/s]



21it [00:00, 204.56it/s]



42it [00:00, 196.79it/s]



62it [00:00, 196.07it/s]



82it [00:00, 187.82it/s]



112it [00:00, 184.15it/s]


671


In [ ]:
evaluate_recommender(joined, model_preds='recs_contextbert4rec_10')

{'ndcg': 0.16966195397762943, 'recall': 0.036534285168058615}

In [ ]:
evaluate_recommender(joined, model_preds='recs_contextbert4rec_5')

{'ndcg': 0.11107867465650255, 'recall': 0.020499472531496678}

In [ ]:
path = r'data/results/contextbert4rec_ml100k.txt'
joined.to_csv(path, index=None, sep=';')

### ML-1m

In [ ]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
df = pd.read_table('data/ratings.dat', sep='::',header=None, names=rnames, engine='python')
df = df.rename(columns={'userId': 'user_id', 'movie_id': 'item_id'})
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df['weekday'] = pd.to_datetime(df.timestamp).dt.weekday
df['hour'] = pd.to_datetime(df.timestamp).dt.hour
df.head()

,user_id,item_id,rating,timestamp,weekday,hour
0,1,1193,5,2000-12-31 22:12:40,6,22
1,1,661,3,2000-12-31 22:35:09,6,22
2,1,914,3,2000-12-31 22:32:48,6,22
3,1,3408,4,2000-12-31 22:04:35,6,22
4,1,2355,5,2001-01-06 23:38:11,5,23


In [ ]:
splitter = RandomSplit(test_fraction=0.2)
train_df, valid_df, test_df = splitter(df)

In [ ]:
train_grouped = train_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id, 
                                                                 x.timestamp,
                                                                 x.weekday,
                                                                 x.hour), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


train_grouped.head()

,user_id,train_interactions
0,1,"[(3186, 2000-12-31 22:00:19, 6, 22), (1270, 20..."
1,2,"[(1198, 2000-12-31 21:28:44, 6, 21), (1210, 20..."
2,3,"[(593, 2000-12-31 21:10:18, 6, 21), (2858, 200..."
3,4,"[(1210, 2000-12-31 20:18:44, 6, 20), (1097, 20..."
4,5,"[(2717, 2000-12-31 05:37:52, 6, 5), (908, 2000..."


In [ ]:
joined = train_grouped.merge(valid_grouped).merge(test_grouped)
joined.head()

,user_id,train_interactions,valid_interactions,test_interactions
0,1,"[(3186, 2000-12-31 22:00:19, 6, 22), (1270, 20...","[(2791, 2000-12-31 22:36:28, 6, 22), (2321, 20...","[(2687, 2001-01-06 23:37:48, 5, 23), (745, 200..."
1,2,"[(1198, 2000-12-31 21:28:44, 6, 21), (1210, 20...","[(2028, 2000-12-31 21:56:13, 6, 21), (2571, 20...","[(1372, 2000-12-31 21:59:01, 6, 21), (1552, 20..."
2,3,"[(593, 2000-12-31 21:10:18, 6, 21), (2858, 200...","[(648, 2000-12-31 21:24:27, 6, 21), (2735, 200...","[(1270, 2000-12-31 21:30:31, 6, 21), (1079, 20..."
3,4,"[(1210, 2000-12-31 20:18:44, 6, 20), (1097, 20...","[(2947, 2000-12-31 20:23:50, 6, 20), (1214, 20...","[(1240, 2000-12-31 20:24:20, 6, 20), (2951, 20..."
4,5,"[(2717, 2000-12-31 05:37:52, 6, 5), (908, 2000...","[(2323, 2000-12-31 06:50:45, 6, 6), (272, 2000...","[(1715, 2000-12-31 06:58:11, 6, 6), (1653, 200..."


In [ ]:
our_items = set()
for idx, row in tqdm(joined.iterrows()):
    for el in row.train_interactions:
        our_items.add(el[0])
        
len(our_items)

6040it [00:00, 41725.36it/s]


3636

In [ ]:
item2idx = {k: i for i, k in enumerate(our_items)}
idx2item = {i: k for k, i in item2idx.items()}

In [ ]:
n_items = len(item2idx)

train = MyDataset(ds=joined,
                  num_items=n_items, 
                  item2idx=item2idx,
                  phase='train')

valid = MyDataset(ds=joined,
                  num_items=n_items,
                  item2idx=item2idx,
                  phase='valid')

print(len(train),len(valid))

6040 6040


In [ ]:
loaders = {
        "train": DataLoader(train, batch_size=256, collate_fn=collate_fn_train),
        "valid": DataLoader(valid, batch_size=256, collate_fn=collate_fn_valid),
}

In [ ]:
model = ContextBERT4Rec(n_items=len(item2idx)+1, mask_ratio=0.2, hidden_size=64)

optimizer = optim.Adam(model.parameters(), lr=0.001)
lr_scheduler = StepLR(optimizer, step_size=20, gamma=0.1)
engine = dl.DeviceEngine('cpu')
hparams = {
    "anneal_cap": 0.2,
    "total_anneal_steps": 6000,
}


callbacks = [
    dl.NDCGCallback("logits", "targets", [20]),
    dl.MAPCallback("logits", "targets", [10]),
    dl.OptimizerCallback("loss", accumulation_steps=1),
    dl.EarlyStoppingCallback(
        patience=5, loader_key="valid", metric_key="map10", minimize=False
    )
]


runner = RecSysRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    engine=engine,
    hparams=hparams,
    scheduler=lr_scheduler,
    loaders=loaders,
    num_epochs=100,
    verbose=True,
    timeit=True,
    callbacks=callbacks,
)


test_dataset = MyDataset(ds=joined, num_items=n_items, phase='test',item2idx=item2idx)


inference_loader = DataLoader(test_dataset, 
                              batch_size=joined.shape[0]//100, 
                              collate_fn=collate_fn_train,)

preds = []

for prediction in tqdm(runner.predict_loader(loader=inference_loader)):
    preds.extend(prediction.detach().cpu().numpy().tolist())
    
print(len(preds))
assert len(preds) == joined.shape[0]

joined['preds_contextbert4rec'] = preds
joined['recs_contextbert4rec_10'] = joined['preds_contextbert4rec'].apply(lambda x: np.argsort(-np.array(x))[:10])
joined['recs_contextbert4rec_10'] = joined['recs_contextbert4rec_10'].apply(lambda x: [idx2item[t-1] for t in x])
joined['recs_contextbert4rec_5'] = joined['preds_contextbert4rec'].apply(lambda x: np.argsort(-np.array(x))[:5])
joined['recs_contextbert4rec_5'] = joined['recs_contextbert4rec_5'].apply(lambda x: [idx2item[t-1] for t in x])
joined.drop(['preds_contextbert4rec'],axis=1, inplace=True)


1/100 * Epoch (train): 100%|██████████| 24/24 [00:23<00:00,  1.01it/s, _timer/_fps=196.594, _timer/batch_time=0.773, _timer/data_time=0.441, _timer/model_time=0.332, loss=7.475, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]


train (1/100) loss: 7.816615366777836 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


1/100 * Epoch (valid): 100%|██████████| 24/24 [00:11<00:00,  2.05it/s, _timer/_fps=550.620, _timer/batch_time=0.276, _timer/data_time=0.022, _timer/model_time=0.254, loss=7.439, lr=1.000e-03, map10=0.090, momentum=0.900, ndcg20=0.053]


valid (1/100) loss: 7.4524627647652535 | lr: 0.001 | map10: 0.12186652710106198 | map10/std: 0.015170085512908977 | momentum: 0.9 | ndcg20: 0.07060247986817991 | ndcg20/std: 0.007212493518638942
* Epoch (1/100) 


2/100 * Epoch (train): 100%|██████████| 24/24 [00:22<00:00,  1.07it/s, _timer/_fps=249.658, _timer/batch_time=0.609, _timer/data_time=0.324, _timer/model_time=0.285, loss=7.323, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]


...

train (55/100) loss: 5.932976117670932 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


55/100 * Epoch (valid): 100%|██████████| 24/24 [00:10<00:00,  2.24it/s, _timer/_fps=516.875, _timer/batch_time=0.294, _timer/data_time=0.023, _timer/model_time=0.271, loss=5.701, lr=1.000e-03, map10=0.248, momentum=0.900, ndcg20=0.169]


valid (55/100) loss: 5.7274621325612864 | lr: 0.001 | map10: 0.2736841744539754 | map10/std: 0.026561318243751676 | momentum: 0.9 | ndcg20: 0.18314779846873508 | ndcg20/std: 0.021522749478485998
* Epoch (55/100) 


101it [00:05, 17.99it/s]


6040


In [ ]:
evaluate_recommender(joined, model_preds='recs_contextbert4rec_10')

{'ndcg': 0.1536313554554575, 'recall': 0.0365331557421488}

In [ ]:
evaluate_recommender(joined, model_preds='recs_contextbert4rec_5')

{'ndcg': 0.09266108158521663, 'recall': 0.019501831360733798}

In [ ]:
path = r'data/results/contextbert4rec_ml1m.txt'
joined.to_csv(path, index=None, sep=';')

### ML-20m

In [10]:
df = pd.read_csv('ML_20m.csv')
df = df.rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df['weekday'] = pd.to_datetime(df.timestamp).dt.weekday
df['hour'] = pd.to_datetime(df.timestamp).dt.hour
df.head()

,user_id,item_id,rating,timestamp,weekday,hour
0,1,2,3.5,2005-04-02 23:53:47,5,23
1,1,29,3.5,2005-04-02 23:31:16,5,23
2,1,32,3.5,2005-04-02 23:33:39,5,23
3,1,47,3.5,2005-04-02 23:32:07,5,23
4,1,50,3.5,2005-04-02 23:29:40,5,23


In [11]:
splitter = RandomSplit(test_fraction=0.3)
train_df, valid_df, test_df = splitter(df)

In [12]:
train_grouped = train_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id, 
                                                                 x.timestamp,
                                                                 x.weekday,
                                                                 x.hour), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


train_grouped.head()

,user_id,train_interactions
0,1,"[(924, 2004-09-10 03:06:38, 4, 3), (919, 2004-..."
1,2,"[(62, 2000-11-21 15:29:58, 1, 15), (469, 2000-..."
2,3,"[(589, 1999-12-11 07:25:08, 5, 7), (1188, 1999..."
3,4,"[(380, 1996-08-24 09:27:05, 5, 9), (165, 1996-..."
4,5,"[(17, 1996-12-25 15:15:35, 2, 15), (62, 1996-1..."


In [13]:
joined = train_grouped.merge(valid_grouped).merge(test_grouped)
joined.head()

,user_id,train_interactions,valid_interactions,test_interactions
0,1,"[(924, 2004-09-10 03:06:38, 4, 3), (919, 2004-...","[(1333, 2005-04-02 23:36:30, 5, 23), (2644, 20...","[(31696, 2005-04-02 23:49:08, 5, 23), (2193, 2..."
1,2,"[(62, 2000-11-21 15:29:58, 1, 15), (469, 2000-...","[(242, 2000-11-21 15:32:56, 1, 15), (1673, 200...","[(1544, 2000-11-21 15:35:43, 1, 15), (1971, 20..."
2,3,"[(589, 1999-12-11 07:25:08, 5, 7), (1188, 1999...","[(2371, 1999-12-11 13:28:35, 5, 13), (1097, 19...","[(2009, 1999-12-14 12:48:50, 1, 12), (2640, 19..."
3,4,"[(380, 1996-08-24 09:27:05, 5, 9), (165, 1996-...","[(350, 1996-08-24 09:31:27, 5, 9), (440, 1996-...","[(520, 1996-08-24 09:34:42, 5, 9), (733, 1996-..."
4,5,"[(17, 1996-12-25 15:15:35, 2, 15), (62, 1996-1...","[(260, 1996-12-25 15:26:48, 2, 15), (318, 1996...","[(377, 1996-12-25 15:28:43, 2, 15), (454, 1996..."


In [14]:
our_items = set()
for idx, row in tqdm(joined.iterrows()):
    for el in row.train_interactions:
        our_items.add(el[0])
        
len(our_items)

8206it [00:00, 17131.61it/s]


10687

In [15]:
item2idx = {k: i for i, k in enumerate(our_items)}
idx2item = {i: k for k, i in item2idx.items()}

In [16]:
n_items = len(item2idx)

train = MyDataset(ds=joined,
                  num_items=n_items, 
                  item2idx=item2idx,
                  phase='train')

valid = MyDataset(ds=joined,
                  num_items=n_items,
                  item2idx=item2idx,
                  phase='valid')

print(len(train),len(valid))

8206 8206


In [17]:
loaders = {
        "train": DataLoader(train, batch_size=256, collate_fn=collate_fn_train),
        "valid": DataLoader(valid, batch_size=256, collate_fn=collate_fn_valid),
}

In [19]:

model = ContextBERT4Rec(n_items=len(item2idx)+1, mask_ratio=0.2, hidden_size=64)

optimizer = optim.Adam(model.parameters(), lr=0.001)
lr_scheduler = StepLR(optimizer, step_size=20, gamma=0.1)
engine = dl.DeviceEngine('cpu')
hparams = {
    "anneal_cap": 0.2,
    "total_anneal_steps": 6000,
}


callbacks = [
    dl.NDCGCallback("logits", "targets", [20]),
    dl.MAPCallback("logits", "targets", [10]),
    dl.OptimizerCallback("loss", accumulation_steps=1),
    dl.EarlyStoppingCallback(
        patience=5, loader_key="valid", metric_key="map10", minimize=False
    )
]


runner = RecSysRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    engine=engine,
    hparams=hparams,
    scheduler=lr_scheduler,
    loaders=loaders,
    num_epochs=100,
    verbose=True,
    timeit=True,
    callbacks=callbacks,
)


1/100 * Epoch (train):   0%|          | 0/33 [00:00<?, ?it/s]

train (1/100) loss: 8.496378980553388 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


1/100 * Epoch (valid):   0%|          | 0/33 [00:00<?, ?it/s]

valid (1/100) loss: 7.777362663804572 | lr: 0.001 | map10: 0.14799357819217254 | map10/std: 0.012865003862810535 | momentum: 0.9 | ndcg20: 0.09101858137294834 | ndcg20/std: 0.007402617064773018
* Epoch (1/100) 


2/100 * Epoch (train):   0%|          | 0/33 [00:00<?, ?it/s]

...

valid (7/100) loss: 7.430047239526261 | lr: 0.001 | map10: 0.18308867126260187 | map10/std: 0.016741962075944316 | momentum: 0.9 | ndcg20: 0.12042275967465474 | ndcg20/std: 0.009436161832236842
* Epoch (7/100) 


8/100 * Epoch (train):   0%|          | 0/33 [00:00<?, ?it/s]

train (8/100) loss: 7.440169526517842 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


8/100 * Epoch (valid):   0%|          | 0/33 [00:00<?, ?it/s]

valid (8/100) loss: 7.3662638084324215 | lr: 0.001 | map10: 0.1884498508375155 | map10/std: 0.015223295459122996 | momentum: 0.9 | ndcg20: 0.1229966996342503 | ndcg20/std: 0.01039137608506247
* Epoch (8/100) 


In [20]:
test_dataset = MyDataset(ds=joined, num_items=n_items, phase='test',item2idx=item2idx)


inference_loader = DataLoader(test_dataset, 
                              batch_size=joined.shape[0]//100, 
                              collate_fn=collate_fn_train,)

preds = []

for prediction in tqdm(runner.predict_loader(loader=inference_loader, engine=dl.DeviceEngine('cpu'))):
    preds.extend(prediction.detach().cpu().numpy().tolist())
    
print(len(preds))
assert len(preds) == joined.shape[0]

joined['preds_contextbert4rec'] = preds
joined['recs_contextbert4rec_10'] = joined['preds_contextbert4rec'].apply(lambda x: np.argsort(-np.array(x))[:10])
joined['recs_contextbert4rec_10'] = joined['recs_contextbert4rec_10'].apply(lambda x: [idx2item[t-1] for t in x])
joined['recs_contextbert4rec_5'] = joined['preds_contextbert4rec'].apply(lambda x: np.argsort(-np.array(x))[:5])
joined['recs_contextbert4rec_5'] = joined['recs_contextbert4rec_5'].apply(lambda x: [idx2item[t-1] for t in x])
joined.drop(['preds_contextbert4rec'],axis=1, inplace=True)
joined.head()

101it [00:19,  5.22it/s]


8206


,user_id,train_interactions,valid_interactions,test_interactions,recs_contextbert4rec_10,recs_contextbert4rec_5
0,1,"[(924, 2004-09-10 03:06:38, 4, 3), (919, 2004-...","[(1333, 2005-04-02 23:36:30, 5, 23), (2644, 20...","[(31696, 2005-04-02 23:49:08, 5, 23), (2193, 2...","[2571, 2858, 1210, 356, 858, 527, 480, 2028, 1...","[2571, 2858, 1210, 356, 858]"
1,2,"[(62, 2000-11-21 15:29:58, 1, 15), (469, 2000-...","[(242, 2000-11-21 15:32:56, 1, 15), (1673, 200...","[(1544, 2000-11-21 15:35:43, 1, 15), (1971, 20...","[2571, 260, 593, 296, 1198, 356, 527, 318, 499...","[2571, 260, 593, 296, 1198]"
2,3,"[(589, 1999-12-11 07:25:08, 5, 7), (1188, 1999...","[(2371, 1999-12-11 13:28:35, 5, 13), (1097, 19...","[(2009, 1999-12-14 12:48:50, 1, 12), (2640, 19...","[296, 480, 356, 2571, 527, 110, 780, 2858, 608...","[296, 480, 356, 2571, 527]"
3,4,"[(380, 1996-08-24 09:27:05, 5, 9), (165, 1996-...","[(350, 1996-08-24 09:31:27, 5, 9), (440, 1996-...","[(520, 1996-08-24 09:34:42, 5, 9), (733, 1996-...","[153, 349, 592, 316, 590, 434, 344, 588, 292, ...","[153, 349, 592, 316, 590]"
4,5,"[(17, 1996-12-25 15:15:35, 2, 15), (62, 1996-1...","[(260, 1996-12-25 15:26:48, 2, 15), (318, 1996...","[(377, 1996-12-25 15:28:43, 2, 15), (454, 1996...","[296, 592, 588, 165, 344, 349, 153, 316, 318, ...","[296, 592, 588, 165, 344]"


In [21]:
evaluate_recommender(joined, model_preds='recs_contextbert4rec_10')

{'ndcg': 0.18479868537510605, 'recall': 0.03895265413022339}

In [22]:
evaluate_recommender(joined, model_preds='recs_contextbert4rec_5')

{'ndcg': 0.11647701605914959, 'recall': 0.020529163681027914}